In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow_probability as tfp
from scipy.interpolate import RectBivariateSpline
import numpy as np
import matplotlib.pyplot as plt
import corner
import importlib
import logging
from tensorflow import keras

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

importlib.reload(logging)

# limit GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=20000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    print(e)

2 Physical GPUs, 1 Logical GPU


2022-05-02 15:17:33.015007: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 15:17:33.540709: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20000 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
dataset_number = 1
training_data = np.load("./nsi_data/sample_nsi_regression_1e7_v{}.npz".format(dataset_number))
data_all = np.column_stack([training_data['ve_dune'][:,:36], training_data['vu_dune'][:,:36], training_data['vebar_dune'][:,:36], training_data['vubar_dune'][:,:36]])

# theta13, theta23, delta, mumu, emu, etau
target = np.column_stack([training_data["theta13"]/180, training_data["theta23"]/180,
                          np.sin(training_data["delta"]/180*np.pi), np.cos(training_data["delta"]/180*np.pi),
                         training_data["mumu"], training_data["emu"],
                         training_data["etau"]])

x_train = data_all[:90]
y_train = target[:90]

x_train_poisson = np.random.poisson(x_train)/1000

In [4]:
gaussian_number = 10
latent_dim = 4
latent_dim_2 = 7
x_parameter_node = [2048, 2048, 2048]
x_spectrum_node = [2048, 2048, 2048]
x_latent_node = [2048, 2048, 2048]
kl_scaling = 10000.0
lr = 2e-05


In [5]:
path = './CVAE/model_{}'.format(model_index)
encoder1 = load_model(path + "/encoder_1.h5", compile=False)
encoder2 = load_model(path + "/encoder_2.h5", compile=False)
decoder = load_model(path + "/decoder.h5", compile=False)

In [15]:
class CVAE(keras.Model):
    def __init__(self, encoder1, encoder2, decoder, **kwargs):
        super(CVAE, self).__init__(**kwargs)
        self.encoder1 = encoder1
        self.encoder2 = encoder2
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    
    def evaluate(self, data):
        x, y = data
        SMALL_CONSTANT = 1e-12
        
        z1_mean, z1_log_var = self.encoder1(x)
        
        temp_var_q = SMALL_CONSTANT + tf.exp(z1_log_var)
        mvn_q = tfp.distributions.MultivariateNormalDiag(
                        loc=z1_mean,
                        scale_diag=tf.sqrt(temp_var_q))
        
        z1 = mvn_q.sample()
        
        z2_mean, z2_log_var, z2_weight = self.encoder2(x[1])

        z2_mean = tf.reshape(z2_mean, (-1, gaussian_number, latent_dim))
        z2_log_var = tf.reshape(z2_log_var, (-1, gaussian_number, latent_dim))
        z2_weight = tf.reshape(z2_weight, (-1, gaussian_number))
        
        temp_var_r1 = SMALL_CONSTANT + tf.exp(z2_log_var)
        bimix_gauss = tfp.distributions.MixtureSameFamily(
                        mixture_distribution=tfp.distributions.Categorical(logits=z2_weight),
                        components_distribution=tfp.distributions.MultivariateNormalDiag(
                        loc=z2_mean,
                        scale_diag=tf.sqrt(temp_var_r1)))
        
        reconstruction_mean, reconstruction_var = self.decoder([z1, x[1]])     
        
        temp_var_r2 = SMALL_CONSTANT + tf.exp(reconstruction_var)
        reconstruction_parameter = tfp.distributions.MultivariateNormalDiag(
                                    loc=reconstruction_mean,
                                    scale_diag= tf.sqrt(temp_var_r2))

        kl_loss = tf.reduce_mean(mvn_q.log_prob(z1) - bimix_gauss.log_prob(z1))*kl_scaling
        reconstruction_loss = -1.0*tf.reduce_mean(reconstruction_parameter.log_prob(y))
        
        total_loss = reconstruction_loss + kl_loss
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [16]:
cvae = CVAE(encoder1, encoder2, decoder)
cvae.compile(optimizer=keras.optimizers.Adam())

In [17]:
cvae.evaluate(data = [[y_train, x_train_poisson], y_train])

{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=1826.0387>,
 'reconstruction_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1829.6212>,
 'kl_loss': <tf.Tensor: shape=(), dtype=float32, numpy=-3.582504>}

In [8]:
n = 1
data_poisson_array = x_train[n:n+1]/1000

mean, log_var, weight = encoder2.predict(data_poisson_array)

mean = tf.reshape(mean, (-1, gaussian_number, latent_dim))
log_var = tf.reshape(log_var, (-1, gaussian_number, latent_dim))
weight = tf.reshape(weight, (-1, gaussian_number))

SMALL_CONSTANT = 1e-10
temp_var = SMALL_CONSTANT + tf.exp(log_var)
test_sampling = tfp.distributions.MixtureSameFamily(
              mixture_distribution=tfp.distributions.Categorical(logits=weight),
              components_distribution=tfp.distributions.MultivariateNormalDiag(
              loc=mean,
              scale_diag=tf.sqrt(temp_var)))

sample_amount= 10000
sample_times = 1
prediction = []
for i in range(sample_times):

    Z3 = np.reshape(test_sampling.sample(sample_shape=(sample_amount)), (sample_amount, latent_dim))
    target_array = np.reshape([data_poisson_array for i in range(sample_amount)], (sample_amount, 144))
    reconstruction_mean, reconstruction_var = decoder.predict([Z3, target_array])
    temp_var = SMALL_CONSTANT + tf.exp(reconstruction_var)
    reconstruction_parameter = tfp.distributions.MultivariateNormalDiag(
                            loc=reconstruction_mean,
                            scale_diag= tf.sqrt(temp_var))
    prediction.extend(reconstruction_parameter.sample().numpy())

prediction = np.array(prediction)

In [ ]:
degree = np.angle(prediction[:,3] + 1j*np.array(prediction[:,2]), deg=True)
degree = [d if d > 0 else 360+d for d in degree]
prediction2 = np.column_stack([prediction[:,:2]*180/np.pi, degree, prediction[:, 4:]])
label = ["theta13", "theta23", 'deltacp', 'mumu', 'emu', 'etau']

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(len(label)):
    plt.subplot(3, 2, i+1)
    plt.hist(prediction2[:, i], bins=100)
    plt.xlabel(label[i])
plt.show()

In [ ]:
index = [1,6,11,16,21,7,12,17,22,13,18,23,19,24,25]
k = 0
plt.figure(figsize = (30, 30))
for i in range(len(prediction2[0])):
    for j in range(i+1, len(prediction2[0])):
        plt.subplot(5, 5, index[k])
        plt.hist2d(prediction2[:,i], prediction2[:,j], bins = 100)
        plt.xlabel(label[i])
        plt.ylabel(label[j])
        plt.colorbar()
        k += 1
plt.savefig(path + '/dataset_v{}_{}'.format(dataset_number, n) + ' ML triangular density plot')
plt.show()

In [ ]:
figure = corner.corner(prediction2,
                    levels = [[0.68] for i in range(15)],
                    scale_hist=True,
                    plot_datapoints=False,
                    labels = label,
                    plot_contours = True,
                    plot_density = False,
                    bins=100,
                    smooth=True
                    )
figure.savefig(path + '/dataset_v{}_{}'.format(dataset_number, n) + ' ML triangular contour')


In [ ]:
def plot_graph(x, y, c, i, name):
    plt.scatter(x, y, c=c)
    plt.colorbar()
    plt.title('data{}_chi-square value'.format(i))
    plt.xlabel('$\\theta_{23}$ (degree)')
    plt.ylabel('$\delta_{cp}$ (degree)')
    plt.clim(0,)
    plt.savefig('./graph/dataset_v{}_{}'.format(dataset_number, n) + name)
    plt.show()

In [ ]:
for i in range(n+1, n+2):
    a = np.loadtxt('./chi_theta23_delta_contour/chi_theta23_delta_v{}.dat'.format(i))
    arr = list(zip(*a))
    x0 = sorted(set(arr[0]))
    y0 = sorted(set(arr[1]))
    z0 = np.reshape(arr[2], (81, 37))
    f = RectBivariateSpline(x0, y0, z0)
    xnew = np.linspace(35, 55, 1000)
    ynew = np.linspace(0, 360, 1000)
    znew = f(xnew, ynew)
    mx, my = np.meshgrid(ynew, xnew)
    x = np.reshape(mx, 1000000)
    y = np.reshape(my, 1000000)
    z = np.reshape(znew, 1000000)
    plot_graph(y, x, z, i , 'chi-square value_full (RectBivariateSpline)')
    loc = np.where(z < 5.99)
    plot_graph(y[loc], x[loc], z[loc], i, 'chi-square value_two_sigma (RectBivariateSpline)')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,15))
plot_axis = plt.subplot(2,2,1)
plot_axis = plt.subplot(2,2,4)
plot_axis = plt.subplot(2,2,3)
r1 = corner.hist2d(prediction2[:, 1], prediction2[:, 2],
                    levels=(0.95,),
                    scale_hist=True,
                    plot_datapoints=False,
                    color='green',
                    labels= ["$\\theta_{23} $($^\circ$)", "$\delta_{cp} $($^\circ$)"],
                    # range=[[35,55], [0, 360]],
                    plot_contours = True,
                    plot_density = False,
                    fontsize=30,
                    bins = [200, 200],
                    label_kwargs={"fontsize": 30},
                    smooth=True
                   )
one_sigma_region_boundary = np.where(znew > 5.99, 1, 0 )
r2 = CS_1_sigma = plot_axis.contour(xnew, ynew, one_sigma_region_boundary.T, 0, colors='red', linestyles="-", linewidths=1, label = 'Prediction')
plot_axis.set_xlabel("$\\theta_{23} $($^\circ$)", fontsize = 20)
plot_axis.set_ylabel("$\delta_{cp} $($^\circ$)", fontsize = 20)
r2.collections[0].set_label("GLOBES (1$\sigma$)")
plt.legend(bbox_to_anchor=(2.3, 1.7), ncol=1,fontsize=20, markerscale=4, edgecolor = "w",fancybox=False, framealpha=0)
plt.savefig(path + '/dataset_v{}_{}'.format(dataset_number, n) + 'theta23 vs deltacp')
plt.show()